In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from src.data import Data
from src.plothelp import *
import time

Simple Example of tensorflow opearation flow

In [3]:
import tensorflow as tf
a = tf.add(5, 5) #node that adds two numbers is created but not executed
print(a)

Tensor("Add:0", shape=(), dtype=int32)


In [4]:
with tf.Session() as sess:
  writer = tf.summary.FileWriter('./graphs', sess.graph) # for displaying graph in tensorboard
  res = sess.run(a) # executes the node in the session
  print (res)

10


### Load and pre-process the dataset:


In [5]:
data = Data("img")
data.load_pickle("100x100.pickle ")
X_train, X_test, y_train, y_test = data.train_test_split()

In [6]:
X_trains = np.asarray(X_train).reshape((len(X_train), -1))
X_tests = np.asarray(X_test).reshape((len(X_test), -1))
y_trains = np.asarray(y_train)
y_tests = np.asarray(y_test)
print(X_trains.shape, X_tests.shape, y_trains.shape, y_tests.shape)

(4698, 30000) (522, 30000) (4698,) (522,)


In [7]:
y_oh, y_test_oh= data.one_hot(y_trains, y_tests)
y_oh.shape

(4698, 4)

In [8]:
y_oh[0,:] #Example of 1 element in the training data set one-hot-encoded

array([ 1.,  0.,  0.,  0.])

In [9]:
# Split y_train into 54 batches containing 87 img
y_batch = np.array_split(y_oh, 54 )
y_batch = np.asarray(y_batch)
print(y_batch.shape)

(54, 87, 4)


In [10]:
# Split X_test into 54 batches of 87 img
X_train_batch = np.array_split(X_trains, 54 )
X_train_batch = np.asarray(X_train_batch)
print(X_train_batch.shape)

(54, 87, 30000)


In [11]:
#Hyper Paremeters of the model
learning_rate = 0.005 # learning rate of the models
batch_size = 87 # number of images in one training batch
n_epochs = 100 #number of training epochs

In [12]:
X = tf.placeholder(tf.float32, [None, 30000])
Y = tf.placeholder(tf.float32, [None, 4])

In [13]:
w = tf.Variable(tf.random_normal(shape=[30000, 4], stddev=0.01), name="weights")

In [14]:
logits = tf.matmul(X, w)

In [15]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
acc = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
acc = tf.reduce_mean(tf.cast(acc, tf.float32))

init = tf.global_variables_initializer()

In [16]:
saver = tf.train.Saver()

In [17]:
tf.summary.scalar("loss", loss) # Create a summary to monitor cost tensor
tf.summary.scalar("accuracy", acc) # Create a summary to monitor accuracy tensor
merged_summary_op = tf.summary.merge_all() # Merge all summaries into a single op

In [18]:
print("Training on full dataset")
with tf.Session() as sess:
  writer = tf.summary.FileWriter('./graphs', sess.graph)
  sess.run(init)
  for i in range(100):  # train the model n_epochs times
    _, c= sess.run([optimizer, loss], feed_dict={X: X_trains, Y: y_oh})
    
    
    print("Epoch:", '%04d' % (i+1), "cost=", "{:.9f}".format(c), "accuracy= ", acc)
       
  save_path = saver.save(sess, "./models/linear-nobatch.ckpt")
  print("Model saved in file: %s" % save_path)

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-18-7ea30d3aa369>, line 1)

In [28]:
with tf.Session() as sess:
  writer = tf.summary.FileWriter('./graphs', sess.graph)
  sess.run(init)
  batch_img = 54 # number of images in one batch
  for i in range(n_epochs):  # train the model n_epochs times
    avg_cost = 0.
    avg_acc = 0.
    
    for j in range(batch_img):
            X_batch = X_train_batch[j,:]
            Y_batch = y_batch[j,:]
            _, c, summary, a = sess.run([optimizer, loss,merged_summary_op, acc], feed_dict={X: X_batch, Y: Y_batch})
            
            writer.add_summary(summary, i * batch_img + i)
            avg_acc += a/batch_img
            avg_cost += c / batch_img
        
    print("Epoch:", '%03d' % (i+1), "cost=", "{:.9f}".format(avg_cost), "accuracy=",avg_acc)

  save_path = saver.save(sess, "./models/linear-gd.ckpt")
  print("Model saved in file: %s" % save_path)

Epoch: 001 cost= 565709.815556279 accuracy= 0.306726266664
Epoch: 002 cost= 401139.434606481 accuracy= 0.363346104545
Epoch: 003 cost= 349518.865162037 accuracy= 0.394423156701
Epoch: 004 cost= 356296.905526620 accuracy= 0.383567473679
Epoch: 005 cost= 307014.866898148 accuracy= 0.424435933155
Epoch: 006 cost= 296364.200810185 accuracy= 0.429331627709
Epoch: 007 cost= 271533.585503472 accuracy= 0.444444442789
Epoch: 008 cost= 266642.361979167 accuracy= 0.43912302813
Epoch: 009 cost= 273615.852141204 accuracy= 0.439548744648
Epoch: 010 cost= 277616.499855324 accuracy= 0.436355894638
Epoch: 011 cost= 258379.697771991 accuracy= 0.445083013287
Epoch: 012 cost= 275902.160590278 accuracy= 0.44955300043
Epoch: 013 cost= 242522.921006944 accuracy= 0.468922943429
Epoch: 014 cost= 227293.061053241 accuracy= 0.474244360571
Epoch: 015 cost= 244742.041232639 accuracy= 0.46679437878
Epoch: 016 cost= 239569.605758102 accuracy= 0.482332906789
Epoch: 017 cost= 221590.670138889 accuracy= 0.483822899284


In [27]:
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

##for changine optimizer from adam to gradien descent 

In [32]:
with tf.Session() as sess:
  saver.restore(sess, "./models/linear-gd.ckpt")
  print("Model restored.")
  
  _, loss_batch, logits_batch = sess.run([optimizer, loss, logits],
                                         feed_dict={X: X_tests, Y: y_test_oh})
  preds = tf.nn.softmax(logits_batch)
  correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(y_test_oh, 1))
  accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
  correct_preds = sess.run(accuracy)
  print ("Accuracy {0}".format(correct_preds / 522), "Predicted ",correct_preds ," correctly")

Model restored.
Accuracy 0.4444444444444444 Predicted  232.0  correctly
